<a href="https://colab.research.google.com/github/seandavi/notebooks/blob/main/uccc_pubmeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install -q biopython httpx polars spacy openpyxl scikit-learn

In [10]:
import polars as pl

# Load your Polars DataFrame with a 'pmid' column
df = pl.read_csv("uccc_pubmeds.csv", low_memory=False, ignore_errors=True)
df.columns = [x.lower() for x in df.columns]


In [11]:
df

journal_abbr,currentcompositeyrif,annualif,reportyr,pmid,pmc,jrnl,citation,pubyear,member,prog,cpc,dt,mco,thi,coding,sr,sr overall,aisr,bbsr,ctsr,d3sr,fcsr,fgsr,gsr,himsr,mssr,phsr,psr,sbsr,inst,label,reporttype,doi,pmc2,title,journal_nlm,journal_issn,journal_full,journal_vol,journal_issue,journal_pages,author_n,author_list,author_list_aff,author_group,collab_n,collab_list,collab_list_aff,type,status,date_publish,abstract,year3,journal_if4
str,str,f64,i64,i64,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""Int J Radiat Oncol Biol Phys""","""7""",5.554,2021,29395629,"""PMC6102100""","""Int J Radiat Oncol Biol Phys""","""Ohri N, Tomé WA, Méndez Romero…",2021,"""Miften-1596""","""DT, LAPS""",null,"""X""",null,null,null,"""None""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""9""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Nicotine Tob Res""","""4.7""",3.786,2020,29741713,"""PMC6751522""","""Nicotine Tob Res""","""Urman R, McConnell R, Unger JB…",2019,"""Samet-1979""","""CPC-25""","""Y""",null,null,null,null,"""none""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""9""",null,null,null,null,null,null,null,"""Nicotine & Tobacco Research""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Ann Surg""","""10.1""",9.203,2020,29771723,null,"""Ann Surg""","""Torphy RJ, Friedman C, Halpern…",2019,"""McCarter-1354, Schulick-1932""","""DT-4""",null,"""Y""",null,null,null,"""none""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""9""",null,null,null,null,null,null,null,"""ANNALS OF SURGERY""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""J Pediatr Hematol Oncol""","""1.2""",1.06,2020,30095692,"""PMC6661241""","""J Pediatr Hematol Oncol""","""Ross S, Eisenman K, Maloney KW…",2019,"""Maloney-1525""","""DT-1""",null,"""Y""",null,null,null,"""none""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""9""",null,null,null,null,null,null,null,"""JOURNAL OF PEDIATRIC HEMATOLOG…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""J Am Acad Dermatol""","""13.8""",6.898,2020,30296542,"""PMC6661165""","""J Am Acad Dermatol""","""Altieri L, Eguchi M, Peng DH, …",2019,"""Cockburn-1861""","""CPC-25""","""Y""",null,null,null,null,"""none""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""9""",null,null,null,null,null,null,null,"""JOURNAL OF THE AMERICAN ACADEM…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Surgery""","""3.8""",3.8,2023,37391327,null,"""Surgery""","""Stuart CM, Kirsch MJ, Zhuang Y…",2023,"""Lieu-1716, Messersmith-1552, S…","""DT""",null,"""X""",null,null,"""0001, 0501""","""None""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""9""","""None""","""01-Report""",null,null,null,null,"""0039-6060""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Transplant Cell Ther""","""3.2""",3.2,2023,37394115,"""PMC10529970""","""Transplant Cell Ther""","""Holland EM, Yates B, Steinberg…",2023,"""Fry-1924""","""THI""",null,null,null,"""X""","""0000, 0004, 1300, 1304""","""None""",null,null,null,null,null,null,null,null,null,null,null,null,null,"""9""","""None""","""01-Report""",null,null,null,null,"""2666-6367""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Proc Natl Acad Sci U S A""","""11.1""",11.1,2023,37406303,"""PMC9963276""","""Proc Natl Acad Sci U S A""","""Yuita H, López-Moyado IF, Jeon…",2023,"""ScottBrowne-1948""","""THI""",null,null,null,"""X""","""0000, 1300""","""None""

In [12]:
from Bio import Entrez
from Bio import Medline

# Replace with your actual email address (required by Entrez)
Entrez.email = "seandavi@gmail.com"

def fetch_pubmed_records(pmids):
    """
    Fetches PubMed records for a list of PMIDs.

    Args:
        pmids: A list of PMIDs (strings).

    Returns:
        A list of PubMed records as dictionaries.
    """
    # Fetch PubMed records in chunks of size 100
    records = []
    for i in range(0, len(pmids), 100):
        chunk = pmids[i:i + 100]
        handle = Entrez.efetch(db="pubmed", id=",".join(chunk), rettype="medline", retmode="text")
        records.extend(list(Medline.parse(handle)))

    return records


In [20]:
# Extract PMIDs as a list of strings
pmids_list = df['pmid'].cast(str).to_list()

# Fetch PubMed records
pubmed_records = fetch_pubmed_records(pmids_list)


In [21]:
pubmed_records[0]

{'PMID': '29395629',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20210804',
 'LR': '20220508',
 'IS': '1879-355X (Electronic) 0360-3016 (Print) 0360-3016 (Linking)',
 'VI': '110',
 'IP': '1',
 'DP': '2021 May 1',
 'TI': 'Local Control After Stereotactic Body Radiation Therapy for Liver Tumors.',
 'PG': '188-195',
 'LID': 'S0360-3016(17)34525-X [pii] 10.1016/j.ijrobp.2017.12.288 [doi]',
 'AB': 'PURPOSE: To quantitatively evaluate published experiences with hepatic stereotactic body radiation therapy (SBRT), to determine local control rates after treatment of primary and metastatic liver tumors and to examine whether outcomes are affected by SBRT dosing regimen. METHODS AND MATERIALS: We identified published articles that reported local control rates after SBRT for primary or metastatic liver tumors. Biologically effective doses (BEDs) were calculated for each dosing regimen using the linear-quadratic equation. We excluded series in which a wide range of BEDs was used. Individual lesion

In [24]:

# Now you have a list of PubMed records!
# You can access the data like this:
records = []
import pydantic
from pydantic import BaseModel
from typing import Optional

class PubMedRecord(BaseModel):
    title: str
    abstract: Optional[str]
    pmid: str
    pmc: Optional[str]
    journal_title: str
    publication_date: str
    authors: list[str]
    submitted_authors: list[str]
    affiliations: list[str]

for record in pubmed_records:
    # Create a PubMedRecord instance for each record
    pubmed_record = PubMedRecord(
        title=record.get('TI', None),
        abstract=record.get('AB', None),
        pmid=record.get('PMID', None),
        pmc=record.get('PMC', None),
        journal_title=record.get('JT', None),
        publication_date=record.get('DP', None),
        authors=record.get('AU', []),
        submitted_authors=record.get('FAU', []),
        affiliations=record.get('AD', [])
    )
    records.append(pubmed_record)

In [25]:
pubmed_df = pl.DataFrame(records)

In [26]:
pubmed_df

title,abstract,pmid,pmc,journal_title,publication_date,authors,submitted_authors,affiliations
str,str,str,str,str,str,list[str],list[str],list[str]
"""Local Control After Stereotact…","""PURPOSE: To quantitatively eva…","""29395629""","""PMC6102100""","""International journal of radia…","""2021 May 1""","[""Ohri N"", ""Tome WA"", … ""Jackson A""]","[""Ohri, Nitin"", ""Tome, Wolfgang A"", … ""Jackson, Andrew""]","[""Department of Radiation Oncology, Montefiore Medical Center and Albert Einstein College of Medicine, Bronx, New York."", ""Department of Radiation Oncology, Montefiore Medical Center and Albert Einstein College of Medicine, Bronx, New York. Electronic address: wtome@montefiore.org."", … ""Department of Radiation Oncology, Memorial Sloan-Kettering Cancer Center, New York, New York.""]"
"""Electronic Cigarette and Cigar…","""BACKGROUND: A supportive youth…","""29741713""","""PMC6751522""","""Nicotine & tobacco research : …","""2019 Sep 19""","[""Urman R"", ""McConnell R"", … ""Barrington-Trimis JL""]","[""Urman, Robert"", ""McConnell, Rob"", … ""Barrington-Trimis, Jessica L""]","[""Department of Preventive Medicine, Keck School of Medicine, University of Southern California, Los Angeles, CA."", ""Department of Preventive Medicine, Keck School of Medicine, University of Southern California, Los Angeles, CA."", … ""Department of Preventive Medicine, Keck School of Medicine, University of Southern California, Los Angeles, CA.""]"
"""Comparing Short-term and Oncol…","""OBJECTIVE: To compare short-te…","""29771723""",null,"""Annals of surgery""","""2019 Dec""","[""Torphy RJ"", ""Friedman C"", … ""Gleisner A""]","[""Torphy, Robert J"", ""Friedman, Chloe"", … ""Gleisner, Ana""]","[""Department of Surgery, University of Colorado, Aurora, CO."", ""Department of Surgery, University of Colorado, Aurora, CO."", … ""Department of Surgery, University of Colorado, Aurora, CO.""]"
"""Successful Use of EPOCH-R in 2…","""Pediatric Burkitt lymphoma has…","""30095692""","""PMC6661241""","""Journal of pediatric hematolog…","""2019 Aug""","[""Ross S"", ""Eisenman K"", ""Maloney KW""]","[""Ross, Savannah"", ""Eisenman, Kristen"", ""Maloney, Kelly W""]","[""Department of Pediatric Hematology/Oncology/BMT, University of Colorado School of Medicine and Children's Hospital Colorado, Aurora, CO.""]"
"""Predictors of mucosal melanoma…","""BACKGROUND: Mucosal melanomas …","""30296542""","""PMC6661165""","""Journal of the American Academ…","""2019 Jul""","[""Altieri L"", ""Eguchi M"", … ""Cockburn M""]","[""Altieri, Lisa"", ""Eguchi, Megan"", … ""Cockburn, Myles""]","[""Department of Dermatology, Keck School of Medicine of the University of Southern California, Los Angeles, California. Electronic address: lisa.altieri@med.usc.edu."", ""University of Colorado Cancer Center, Denver, Colorado."", … ""Department of Dermatology, Keck School of Medicine of the University of Southern California, Los Angeles, California; University of Colorado Cancer Center, Denver, Colorado; Department of Preventive Medicine, Keck School of Medicine of the University of Southern California, Los Angeles, California.""]"
…,…,…,…,…,…,…,…,…
"""Pulmonary metastasectomy is as…","""BACKGROUND: After surgical res…","""37391327""",null,"""Surgery""","""2023 Sep""","[""Stuart CM"", ""Kirsch MJ"", … ""Meguid RA""]","[""Stuart, Christina M"", ""Kirsch, Michael J"", … ""Meguid, Robert A""]","[""Department of Surgery, Division of Cardiothoracic Surgery, University of Colorado School of Medicine, Aurora, CO; Surgical Outcomes and Applied Research Program, Department of Surgery, University of Colorado School of Medicine, Aurora, CO. Electronic address: Christina.Stuart@cuanschutz.edu."", ""Department of Surgery, Division of Surgical Oncology, University of Colorado School of Medicine, Aurora, CO."", … ""Department of Surgery, Division of Cardiothoracic Surgery, University of Colorado School of Medicine, Aurora, CO; Surgical Outcomes and Applied Research Program, Department of Surgery, Un

In [28]:
pubmed_df.shape

(2152, 9)

In [38]:
import spacy
import polars as pl

# Load the spaCy English language model
nlp = spacy.load("en_core_web_sm")

# Define a function to clean the text (same as before)
def clean_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

# Apply the cleaning function using `map` within `select`
cleaned_pubmed_df = pubmed_df.select([
    pl.col("pmid"),
    pl.col("title").map_elements(clean_text, return_dtype=str).alias("cleaned_title"),
    pl.col("abstract").map_elements(clean_text, return_dtype=str).alias("cleaned_abstract"),
])

In [41]:
cleaned_pubmed_df['cleaned_title']

cleaned_title
str
"""local control stereotactic bod…"
"""electronic cigarette cigarette…"
"""compare short term oncologic o…"
"""successful use epoch r young a…"
"""predictor mucosal melanoma sur…"
…
"""pulmonary metastasectomy assoc…"
"""chimeric antigen receptor t ce…"
"""inducible disruption tet gene …"


In [96]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'df' is your Polars DataFrame with 'cleaned_abstract' column
abstracts = cleaned_pubmed_df.filter(
    pl.col("cleaned_abstract").is_not_null()
)['cleaned_abstract'].to_list()

print("Sample abstract:", abstracts[0])

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=100)  # Adjust max_features as needed
tfidfs = vectorizer.fit(abstracts)
tfidf_matrix = vectorizer.fit_transform(abstracts)

# Apply LSA (TruncatedSVD is scikit-learn's implementation of LSA)
lsa = TruncatedSVD(n_components=30)  # Adjust n_components (number of topics)
lsa_matrix = lsa.fit_transform(tfidf_matrix)

# Explore the results
print(lsa.components_)  # Topic-word matrix
#print(lsa_matrix)  # Document-topic matrix

Sample abstract: purpose quantitatively evaluate publish experience hepatic stereotactic body radiation therapy sbrt determine local control rate treatment primary metastatic liver tumor examine outcome affect sbrt dose regiman method material identify publish article report local control rate sbrt primary metastatic liver tumor biologically effective dose bed calculate dose regiman linear quadratic equation exclude series wide range bed individual lesion datum local control extract actuarial survival curve datum aggregate form single dataset actuarial local control curve generate kaplan meier method group lesion disease type bed vs comparison log rank testing result thirteen article meet inclusion criterion form dataset analysis year actuarial local control rate sbrt primary liver tumor n respectively low year actuarial local control rate observe liver metastasis n log rank p patient treat sbrt primary liver tumor evidence local control influence bed range schedule liver metastasis ha

In [97]:
lsa_matrix.shape

(2091, 30)

In [98]:
lsa.components_.shape

(30, 100)

In [99]:
feature_names = tfidfs.get_feature_names_out()

In [100]:
import numpy as np

# Assuming 'lsa' is your trained LSA model and 'feature_names' is from tfidf.get_feature_names_out()

topic_words = []
for i, topic in enumerate(lsa.components_):
    word_value_pairs = list(zip(feature_names, topic))
    sorted_pairs = sorted(word_value_pairs, key=lambda pair: pair[1], reverse=True)
    topic_words.append([word for word, value in sorted_pairs])

In [101]:
res = []
for i in range(30):
    res.append(topic_words[i][0:20])
g = pl.DataFrame(res)

In [102]:
g

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""patient""","""cell""","""cancer""","""tumor""","""dna""","""breast""","""dose""","""patient""","""dna""","""dose""","""care""","""lung""","""mouse""","""protein""","""response""","""mutation""","""risk""","""day""","""intervention""","""ci""","""day""","""year""","""ci""","""month""","""high""","""lung""","""lung""","""use""","""day""","""pathway"""
"""cell""","""tumor""","""breast""","""breast""","""gene""","""cancer""","""care""","""mutation""","""cell""","""risk""","""breast""","""cancer""","""treatment""","""mouse""","""risk""","""mouse""","""ci""","""cancer""","""group""","""use""","""care""","""health""","""gene""","""report""","""low""","""clinical""","""group""","""disease""","""activity""","""use"""
"""cancer""","""expression""","""intervention""","""cancer""","""protein""","""dna""","""mg""","""care""","""tumor""","""breast""","""response""","""risk""","""radiation""","""human""","""immune""","""group""","""protein""","""use""","""activity""","""model""","""model""","""activity""","""disease""","""median""","""treatment""","""risk""","""high""","""intervention""","""lung""","""group"""
"""tumor""","""gene""","""health""","""immune""","""response""","""mutation""","""cancer""","""gene""","""radiation""","""mg""","""intervention""","""dna""","""dna""","""patient""","""use""","""increase""","""care""","""time""","""trial""","""breast""","""disease""","""age""","""human""","""activity""","""mouse""","""trial""","""radiation""","""survival""","""target""","""os"""
"""treatment""","""mouse""","""care""","""radiation""","""tumor""","""woman""","""health""","""identify""","""use""","""high""","""expression""","""mutation""","""expression""","""care""","""disease""","""breast""","""day""","""group""","""patient""","""inhibitor""","""risk""","""report""","""pathway""","""risk""","""human""","""protein""","""rate""","""lung""","""ci""","""care"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""identify""","""line""","""study""","""target""","""find""","""phase""","""use""","""use""","""activity""","""radiation""","""level""","""immune""","""survival""","""primary""","""assess""","""inhibitor""","""target""","""method""","""significant""","""patient""","""identify""","""relate""","""event""","""identify""","""age""","""event""","""primary""","""associate""","""outcome""","""follow"""
"""disease""","""increase""","""age""","""clinical""","""datum""","""dose""","""report""","""compare""","""show""","""mutation""","""year""","""expression""","""combination""","""suggest""","""patient""","""line""","""effect""","""control""","""study""","""activity""","""clinical""","""human""","""control""","""intervention""","""include""","""report""","""respectively""","""identify""","""base""","""response"""
"""ci""","""find""","""increase""","""treat""","""identify""","""patient""","""change""","""target""","""method""","""study""","""ci""","""health""","""breast""","""month""","""target""","""significantly""","""radiation""","""report""","""outcome""","""phase""","""immune""","""combination""","""age""","""event""","""grade""","""tumor""","""provide""","""therapy""","""chemotherapy""","""compare"""


In [103]:
import numpy as np

# Assuming 'lsa_matrix' is your document-topic matrix

top_documents_per_topic = []
for i in range(lsa_matrix.shape[1]):  # Iterate over topics (columns)
    document_value_pairs = list(enumerate(lsa_matrix[:, i]))
    sorted_pairs = sorted(document_value_pairs, key=lambda pair: pair[1], reverse=True)
    top_document_indices = [index for index, value in sorted_pairs[:30]]
    top_documents_per_topic.append(top_document_indices)

In [106]:
for i in cleaned_pubmed_df[top_documents_per_topic[21],:].iter_rows():
    print(i)

('35121993', 'astrocytic drive disseminate breast tumor cell dormancy brain', 'dormancy think play key role metastasis breast tumor cell brain knowledge molecular mechanism regulate disseminate tumor cell dtc dormancy organ limit serial intravital imaging dormant metastatic triple negative breast cancer line identify escape single cell micrometastatic state rate limit step brain metastasis dtc occupy vascular niche quiescent dtcs reside astrocyte endfeet site astrocyte deposit drive dtc quiescence induce dystroglycan receptor associate yes associate protein sequester nucleus prevent prometastatic function finding identify brain specific mechanism dtc dormancy highlight need thorough understanding tumor dormancy develop therapeutic approach prevent brain metastasis')
('33378218', 'suppression alpha catenin adheren junction enhance epithelial cell proliferation motility tace mediate tgf alpha autocrine paracrine signaling', 'sustain cell migration essential wound healing cancer metastasi